# T1 SEMANTIC SEGMENTATION

Import and prepare point clouds for semantic segmentation and do the inference.
To run these scripts, create a python 3.10 environment & install geomapi (numpy, opend3d, ifcopenshell, trimesh, ...), pytorch

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from pathlib import Path
import os
import sys
import numpy as np
import laspy
from geomapi.utils import geometryutils as gmu
import torch
import os
from collections import OrderedDict
import time
import torch.nn.functional as F

#CONTEXT
import context 

#UTILS
from utils.t1_utils import *

#POINTCEPT
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.engines.test import TESTERS
from pointcept.engines.launch import launch
from pointcept.datasets import build_dataset, collate_fn
import pointcept.utils.comm as comm

from pointcept.engines.defaults import create_ddp_model

from pointcept.utils.misc import make_dirs
from pointcept.models import build_model

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## INPUT

In [2]:
path= Path(os.getcwd()).parents[0]/'data'

point_cloud_path =path/'t1_data'/'input'/'Eva.las'
output_folder =  path/'t1_data'/'input'

config_path = path/'t1_data'/'config.py' #Path(os.getcwd())/'data'/'t1_data'/'config.py'
save_path = path/'t1_data' #Path(os.getcwd())/'data'/'t1_data'
weights = path/'t1_data'/'model'/'model_best.pth' #Path(os.getcwd())/'data'/'t1_data'/'model'/'model_best.pth'

## INPUT DATA CONVERSION

Preprocessing of input data

In [3]:


scene_id = os.path.basename(point_cloud_path)

name, ext = os.path.splitext(scene_id)

if ext not in  [".las", ".laz"]:
    exit()

# Read LAS/LAZ
# populate dict
las = laspy.read(point_cloud_path)
print(list(las.point_format.dimension_names))

pcd = gmu.las_to_pcd(las)
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()

coords = np.stack([las.x, las.y, las.z], axis=1)
# colors = np.stack([las.red, las.green, las.blue], axis=1) # betons
colors = np.zeros((len(las.points), 3)).astype(np.uint8) + 255
normals = np.asarray(pcd.normals)    

save_dict = dict(coord=coords, color=colors, normal=normals, scene_id=scene_id) #, semantic_gt=las.labels.astype(int))

torch.save(save_dict, os.path.join(output_folder, f"{name}.pth"))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue']


## INFERENCE

Inference using Point Transformer V3

In [4]:

def collate_fn(batch):
    return batch

def build_inference_model(cfg):
    model = build_model(cfg.model)
    n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    model = create_ddp_model(
        model.cuda(),
        broadcast_buffers=False,
        find_unused_parameters=cfg.find_unused_parameters,
    )
    if os.path.isfile(cfg.weight):
        checkpoint = torch.load(cfg.weight)
        weight = OrderedDict()
        for key, value in checkpoint["state_dict"].items():
            if key.startswith("module."):
                if comm.get_world_size() == 1:
                    key = key[7:]  # module.xxx.xxx -> xxx.xxx
            else:
                if comm.get_world_size() > 1:
                    key = "module." + key  # xxx.xxx -> module.xxx.xxx
            weight[key] = value
        model.load_state_dict(weight, strict=True)

    else:
        raise RuntimeError("=> No checkpoint found at '{}'".format(cfg.weight))
    return model

def main_worker(cfg):    
    cfg = default_setup(cfg)
    test_dataset = build_dataset(cfg.data.test)

    if comm.get_world_size() > 1:
        test_sampler = torch.utils.data.distributed.DistributedSampler(test_dataset)
    else:
        test_sampler = None
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=cfg.batch_size_test_per_gpu,
        shuffle=False,
        num_workers=cfg.batch_size_test_per_gpu,
        pin_memory=True,
        sampler=test_sampler,
        collate_fn=collate_fn,
    )
    
    model = build_inference_model(cfg)
    model.eval()

    save_path = os.path.join(cfg.save_path, "result")
    make_dirs(save_path)
    
    for idx, data_dict in enumerate(test_loader):
        data_dict = data_dict[0]  # current assume batch size is 1
        fragment_list = data_dict.pop("fragment_list")
        segment = data_dict.pop("segment")
        data_name = data_dict.pop("name")
        pred_save_path = os.path.join(save_path, "{}_pred.npy".format(data_name))

        pred = torch.zeros((segment.size, cfg.data.num_classes)).cuda()
        for i in range(len(fragment_list)):
            fragment_batch_size = 1
            s_i, e_i = i * fragment_batch_size, min(
                (i + 1) * fragment_batch_size, len(fragment_list)
            )
            input_dict = collate_fn(fragment_list[s_i:e_i])[0]
            for key in input_dict.keys():
                if isinstance(input_dict[key], torch.Tensor):
                    input_dict[key] = input_dict[key].cuda(non_blocking=True)
            idx_part = input_dict["index"]            
            with torch.no_grad():
                pred_part = model(input_dict)["seg_logits"]  # (n, k)
                pred_part = F.softmax(pred_part, -1)
                if cfg.empty_cache:
                    torch.cuda.empty_cache()
                bs = 0                
                for be in input_dict["offset"]:
                    pred[idx_part[bs:be], :] += pred_part[bs:be]
                    bs = be        
        pred = pred.max(1)[1].data.cpu().numpy()
        np.save(pred_save_path, pred)

    print("DONE.")

cfg = default_config_parser(str(config_path), {'save_path': str(save_path), 'weight': str(weights)})

launch(
    main_worker,
    num_gpus_per_machine=1,
    num_machines=1,
    machine_rank=0,
    dist_url='auto',
    cfg=(cfg,),
)

KeyError: 'ThreeDomDataset is not in the datasets registry'

## RESULTS

Convert inference result back to .las

In [ ]:
import numpy as np

point_cloud_path = Path(os.getcwd())/'data'/'t1_data'/'input'/'Eva.las'
labels_path = Path(os.getcwd())/'data'/'t1_data'/'result'/'Eva_pred.npy'
output_folder =  Path(os.getcwd())/'data'/'t1_data'/'result'

las = laspy.read(point_cloud_path)

las.add_extra_dim(laspy.ExtraBytesParams(
    name="labels",
    type=np.int32
))

labels = np.load(labels_path)

las["labels"] = labels

las.write(os.path.join(output_folder, "Eva_pred.las"))